# A brief summary of C++ (11, 14, 17, 20)
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [C++11](#C++11)
    * [Uniform Initialization](#uniform-initialization)
    * [The `auto` keyword](#the-auto-keyword)
    * [The `decltype` operator](#the-decltype-operator)
    * [The range-based `for` loop](#the-range-based-for-loop)
    * [The `nullptr` keyword](#the-nullptr-keyword)
    * [Enumeration Classes](#enumeration-classes)
* [C++14](#C++14)
* [C++17](#C++17)
* [C++20](#C++20)

<hr>

*Note:* This writing builds upon <a href="https://github.com/hazemanwer2000/gif-quickies/blob/main/brief_summary_of_cpp.ipynb">*A brief summary of C++ (98)*</a>, and discusses feature introductions in modern standards of the C++ language.

## C++11 <a class="anchor" id="c++11"></a>

### Uniform Initialization <a class="anchor" id="uniform-initialization"></a>

Direct initialization using parentheses, `()`, proved sometimes inconsistent. For example:

In [ ]:
int x();           // Interpreted as a function declaration

C++11 introduces a more consistent form of initialization, called *uniform* initialization, using braces, *{}*.

In [ ]:
int x{};           // Default-initialized to '0'

*Note:* Unlike `()`, `{}` does not allow implicit narrowing of values. For example, implicit conversion of `double` to `float` is prohibted.

Additionally, an immutable list, `initializer_list`, is defined in `<initializer_list>`.

If a constructor that accepts an `initializer_list` object exists, then uniform (or, brace) initialization allows for the initialization of the list implicitly by comma-separated values between the opening and closing braces.

This is similar in syntax to how C-style arrays are initialized.

In [26]:
//%cflags: -I.jupyter

#include <iostream>
#include <vector>
#include <initializer_list>
#include "helpers.h"

template <class T>
class Array {
    std::vector<T> v;
public:
    Array(const std::initializer_list<T> &init) : v(init.begin(), init.end()) {}
    Array(int size) : v(size) {}

    void show() { forwardly(v.begin(), v.end()); }
};

int main() {
    Array<int> arr{1, 2, 3};
    arr.show();
}

1, 2, 3,   


In [ ]:
Array<int> arr{1, 2, 3};               /* Equivalent syntax */

Array<int> arr = {1, 2, 3};

Array<int> arr({1, 2, 3});

In [ ]:
Array<int> arr(5);                     /* Array of size '5', default-initialized 'int's to '0' */

Array<int> arr{5};                     /* Array of size '1', consisting of {5} */

*Note:* When an `initializer_list` constructor is missing, `{}` behave similar to `()`.

### The `auto` keyword <a class="anchor" id="the-auto-keyword"></a>

C++11 introduces the `auto` keyword, that may be used as a type, to be implicitly resolved at compile-time.

In [43]:
#include <iostream>

int main() {
    auto x = 5;
    
    std::cout << x;
}

5

In [45]:
#include <iostream>

auto add(int x, int y) {
    return x + y;
}

int main() {
    std::cout << add(1, 2);
}

3

*Note:* Always use copy initialization with the `auto` keyword. In particular, brace initialization resolves `auto` into `initializer_list`.

The `auto` keyword ignores references and top-level constants, but retains low-level constants.

In [ ]:
#include <iostream>

int main() {
    const int y = 6;       /* 'y' is a top-level constant ('const int') */

    auto x = y;            /* auto resolved to 'int' */
    
    x = 7;                 /* Valid */
}      

In [ ]:
#include <iostream>

int main() {
    const int y = 6;       /* '&y' is a low-level constant ('const int *') */

    auto ptr = &y;         /* auto resolved to 'const int *' */
    
    *ptr = 7;              /* Error: Assignment of read-only location */
}      

*Note:* As an example, `const int` and `int * const` are top-level constants, while `const int *` is a low-level constant.

### The `decltype` operator <a class="anchor" id="the-decltype-operator"></a>

C++11 introduces the `decltype` operator, that accepts an expression, and substitutes for its type at compile-time.

In [48]:
#include <iostream>

int main() {
    decltype(5 + 3.2) val = (5 + 3.2);
    
    std::cout << val;
}

8.2

Unlike the `auto` keyword, the `decltype` operator retains all qualifiers, including references and top-level constants.

In [ ]:
int main() {
    const int x = 5;          /* 'x' is a top-level const */
    
    decltype(x) y = 6;        /* Resolves to 'const int' */
    
    y = 7;                    /* Error: Assignment to read-only variable. */
}

### The `constexpr` keyword <a class="anchor" id="the-constexpr-keyword"></a>

C++11 introduces the `constexpr` keyword, placed before a variable declaration gurantees that it is evaluated at compile-time.

When placed before a function, it gurantees that calls to this function are evaluated at compile-time.

In [70]:
#include <iostream>
                                      /* Before Compilation */
constexpr auto add(int x, int y) {
    return x + y;
}

int main() {
    constexpr int res = add(5, 6);
    
    std::cout << res;
}

11

In [66]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << 11;
}

11

A class with a `constexpr` constructor may be used as a literal.

In [64]:
#include <iostream>
                                      /* Before Compilation */
struct Point {
    int x, y;
    
    constexpr Point(int x, int y) : x(x), y(y) {}
};

int main() {
    constexpr Point p(1, 2);
    
    std::cout << "(" << p.x << ", " << p.y << ")";
}

(1, 2)

In [72]:
#include <iostream>
                                      /* After Compilation */
int main() {
    std::cout << "(" << 1 << ", " << 2 << ")";
}

(1, 2)

### The range-based `for` loop <a class="anchor" id="the-range-based-for-loop"></a>

C++11 introduces a range-based `for` loop, that uses `begin` and `end` members of its post-colon argument.

In [79]:
#include <iostream>
#include <vector>

int main() {
    std::vector<int> v {1, 2, 3, 4, 5};

    for (auto &val : v) {
        std::cout << val << ' ';
    }
}

1 2 3 4 5 

*Note:* It is recommended to refrain from using a range-based `for` loop with a built-in array.

### The `nullptr` keyword <a class="anchor" id="the-nullptr-keyword"></a>

C++11 defines the `nullptr` ke|yword, to replace the ambigious zero-defined `NULL` macro.

In [98]:
//%cflags: -w

#include <iostream>

void foo(long int x) {
    std::cout << "foo(long int)\n";
}

void foo(long int *x) {
    std::cout << "foo(long int *)\n";
}

int main() {
    foo(NULL);
    foo(nullptr);
}

foo(long int)
foo(long int *)


### Enumeration Classes <a class="anchor" id="enumeration-classes"></a>

C++11 introduces *enumeration classes*, as a scoped alternative to plain enumerations.

In [104]:
#include <iostream>

enum class Grade {High, Average, Low};

void tell_me(Grade grade) {
    switch (grade) {
        case Grade::High:
            std::cout << "He got a really high grade!";
            break;
        case Grade::Average:
            std::cout << "He's kind'a average.";
            break;
        case Grade::Low:
            std::cout << "He needs to pass the corrective exam, first!";
            break;
    }
}

int main() {
    tell_me(Grade::High);
}

He got a really high grade!

You may specify a specific integral type to contain values from the `enum class` type, which is `int` by default.

In [ ]:
enum class Grade : char {High, Average, Low};

*Note:* Implicit conversion from `int` to an `enum class` type, and vice versa, is prohibited.